In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import ipaddress
from user_agents import parse
from sklearn.preprocessing import OneHotEncoder

In [2]:
current_dataset = pd.read_csv("../data-processed/imputation-v2.csv")

In [3]:
current_dataset.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERZIPCODE,PLATFORMTYPE,PLATFORM,BOT,BROWSER,DIST
0,1001,0,2,2863464470,6974dcaa-f932-480e-9fb5-c52e20e1393a,97206,3,Windows,False,Chrome,NaN
1,1002,0,2,1104739609,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,22202,3,Mac OS,False,Safari,NaN
2,1003,0,2,1122520854,a698de4b-e200-46dd-b5fb-40402175ae18,83654,3,Mac OS,False,Safari,NaN
3,1004,0,3,2302049782,45967533-75c8-4fbd-a00c-e6ff20447aaa,229114624,3,Windows,False,ChromiumEdge,NaN
4,1005,1,7,2932533091,a17e25be-532d-4cf5-b916-9308c8c3961f,77008,3,iOS,False,Safari,iPhone


USERPLATFORMUID converted into binary

In [4]:
# hash_value = "9c36e90c-caa7-4323-8431-fb84f8293972".replace("-","")

# # Convert hash to binary
# binary_value = bin(int(hash_value, 16))[2:]

# print(binary_value)


def convert_to_bin(value):
    hash_value = value.replace("-","")
    return bin(int(hash_value, 16))[2:]

current_dataset["BINARYUID"] = current_dataset["USERPLATFORMUID"].apply(convert_to_bin)
current_dataset = current_dataset.drop(columns="USERPLATFORMUID")
current_dataset.to_csv("../data-processed/imputation-v3.csv")

PLATFORM encoding

In [5]:
current_dataset["DEVICETYPE"].unique()

array([0, 1, 2, 3])

In [6]:
current_dataset.columns

Index(['ID', 'DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP', 'USERZIPCODE',
       'PLATFORMTYPE', 'PLATFORM', 'BOT', 'BROWSER', 'DIST', 'BINARYUID'],
      dtype='object')

In [7]:
encoder_deviceType = LabelEncoder()
encoder_deviceType.fit(['Desktop', 'Mobile', 'Tablet', 'Unknown']) 
if_deviceType = encoder_deviceType.inverse_transform(current_dataset["DEVICETYPE"])
deviceType_pd = pd.DataFrame({"DEVICETYPE": if_deviceType})
current_dataset["DEVICETYPE2"] = deviceType_pd["DEVICETYPE"]
current_dataset = current_dataset.drop(columns=["DEVICETYPE"])
current_dataset.rename(columns={"DEVICETYPE2": "DEVICETYPE"}, inplace=True)

In [8]:
encoder_platformType = LabelEncoder()
encoder_platformType.fit(['Online Medical Journal', 'Online Learning Portal','Medical News Platform', 'Electronic Health Record'])
if_platformType = encoder_platformType.inverse_transform(current_dataset["PLATFORMTYPE"])
current_dataset["PLATFORMTYPE2"] = if_platformType


In [9]:
current_dataset = current_dataset.drop(columns=["PLATFORMTYPE"])
current_dataset.rename(columns={"PLATFORMTYPE2": "PLATFORMTYPE"}, inplace=True)

one Hot encoding of possible parameter

In [10]:
one_hot_encoded_deviceType = pd.get_dummies(current_dataset["DEVICETYPE"])
one_hot_encoded_platformId = pd.get_dummies(current_dataset["PLATFORM_ID"])
one_hot_encoded_platformType = pd.get_dummies(current_dataset["PLATFORMTYPE"])
one_hot_encoded_platform = pd.get_dummies(current_dataset["PLATFORM"])
one_hot_encoded_bot = pd.get_dummies(current_dataset["BOT"])
one_hot_encoded_browser = pd.get_dummies(current_dataset["BROWSER"])
one_hot_encoded_dist = pd.get_dummies(current_dataset["DIST"])


In [11]:
cols = one_hot_encoded_platformId

for col in cols:
    col_name = "PLATFORMID_"+ str(col)
    one_hot_encoded_platformId = one_hot_encoded_platformId.rename(columns={col: col_name})


In [12]:
one_hot_encoded_platformId

,PLATFORMID_1,PLATFORMID_2,PLATFORMID_3,PLATFORMID_4,PLATFORMID_5,PLATFORMID_6,PLATFORMID_7,PLATFORMID_8,PLATFORMID_9,PLATFORMID_10,PLATFORMID_12,PLATFORMID_13,PLATFORMID_14,PLATFORMID_15
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109333,0,1,0,0,0,0,0,0,0,0,0,0,0,0
109334,0,1,0,0,0,0,0,0,0,0,0,0,0,0
109335,0,0,0,0,0,0,1,0,0,0,0,0,0,0
109336,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
df_encoded = pd.concat([one_hot_encoded_dist,
                        one_hot_encoded_browser, 
                        one_hot_encoded_bot, 
                        one_hot_encoded_platform, 
                        one_hot_encoded_platformType, 
                        one_hot_encoded_platformId, 
                        one_hot_encoded_deviceType], axis=1)

In [14]:
df_encoded["USERZIPCODE"] = current_dataset["USERZIPCODE"]
df_encoded["BIDREQUESTIP"] = current_dataset["BIDREQUESTIP"]


In [15]:
df_encoded.to_csv("../data-processed/model-train-test.csv")

In [18]:
df_encoded.describe()

,Android,IPad,Ubuntu,iPhone,BingBot,Chrome,ChromeiOS,ChromiumEdge,FacebookExternalHit,Firefox,...,PLATFORMID_12,PLATFORMID_13,PLATFORMID_14,PLATFORMID_15,Desktop,Mobile,Tablet,Unknown,USERZIPCODE,BIDREQUESTIP
count,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,...,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,109338.000000,1.093380e+05,1.093380e+05
mean,0.088542,0.024493,0.000220,0.200360,0.000165,0.504253,0.022115,0.094405,0.000064,0.043608,...,0.004966,0.000284,0.006631,0.000384,0.687565,0.281540,0.030730,0.000165,1.830356e+07,1.822784e+09
std,0.284083,0.154574,0.014814,0.400272,0.012830,0.499984,0.147058,0.292392,0.008001,0.204222,...,0.070297,0.016836,0.081160,0.019596,0.463488,0.449752,0.172587,0.012830,1.066942e+08,9.236351e+08
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.020000e+02,3.727291e+07
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.935500e+04,1.163670e+09
50%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4.321000e+04,1.645450e+09
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,7.708975e+04,2.733373e+09
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.970149e+08,3.652516e+09
